In [ ]:
import pandas as pd
import pickle

from collections import defaultdict
import string

import torch
from torch import nn
from torch.nn.functional import softmax
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
from transformers import DebertaConfig
from datasets import Dataset
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

from IPython.display import display, HTML

from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

2025-07-02 01:48:01.614805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751420881.821455      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751420881.878774      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from huggingface_hub import login

login(new_session=False, # Won’t request token if one is already saved on machine
write_permission=True, # Requires a token with write permission
token="", # The name of your token
add_to_git_credential=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)
Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [3]:
checkpoint = "ConditionalNLI/Exp5_deberta-large-mnli"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3, device_map="auto", output_attentions=True)

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/969 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/985 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

In [ ]:
config = DebertaConfig()
batch_size = 4
criterion = nn.CrossEntropyLoss()
config.max_position_embeddings

512

## Running on Dataset

In [6]:
from datasets import load_dataset, concatenate_datasets
import json
import ast

In [7]:
test_df4 = pd.read_excel('/kaggle/input/confer-extension/Part5_Dataset/Part5_Type4.xlsx')
test_df4

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related
...,...,...,...,...,...,...,...
292,"If Christina is a makeup artist, she'll grab h...",Christina has a makeup sponge.,E,possessive,her makeup sponge,rides a scooter,unrelated
293,"If Rahim is a clergyman, he'll bring his holy ...",Rahim has a holy book.,E,possessive,his holy book,gardens roses,unrelated
294,"If James is a police officer, he'll pick his g...",James has a gun.,E,possessive,his gun,drinks tea,unrelated
295,"If Maya is a piano teacher, she'll pack her me...",Maya has a metronome.,E,possessive,her metronome,knits hats,unrelated


In [8]:
test_df5p = pd.read_excel('/kaggle/input/confer-extension/Part5_Dataset/Part5_Type5P.xlsx')
test_df5p

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,the university,related
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,post exam celebratory dinner,related
2,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,her husband,lunch,related
3,"If Chloe finishes his project early, his colle...",Chloe has colleagues.,E,possessive,his colleagues,drinks,related
4,"If Charlotte passes her math test, her parents...",Charlotte has parents.,E,possessive,her parents,a calculator,related
...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated


In [9]:
test_df5a = pd.read_excel('/kaggle/input/confer-extension/Part5_Dataset/Part5_Type5A.xlsx')
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related
...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated


In [10]:
test_df = pd.concat([test_df4,test_df5a,test_df5p])
test_df

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related
2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related
3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related
4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related
...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated


In [11]:
test_df.to_csv('part5_all.csv')

In [12]:
sample_test_df = test_df4.iloc[:8].reset_index()
sample_test_df 

,index,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
1,1,"If Bill is a content strategist, he'll use his...",Bill has a content management application.,E,possessive,his content management application,plan and organize digital content,related
2,2,"If Steve is a conservation officer, he'll use ...",Steve has a wildlife trap.,E,possessive,his wildlife trap,monitor and protect animal populations,related
3,3,"If Matt is a scuba diver, he'll wear his wetsu...",Matt has a wetsuit.,E,possessive,his wetsuit,maintain body temperature underwater,related
4,4,"If Noah is a pastry chef, he'll use his pastry...",Noah has a pastry bag.,E,possessive,his pastry bag,decorate cakes and pastries,related
5,5,"If Donald is a crime scene investigator, he'll...",Donald has evidence bags.,E,possessive,his evidence bags,secure crime scene samples,related
6,6,"If Edward is a playwright, he'll use his pen t...",Edward has a pen.,E,possessive,his pen,write scripts for the stage,related
7,7,"If William is a notary, he'll use his notary s...",William has a notary seal.,E,possessive,his notary seal,certify documents,related


In [13]:
sample_test_df = test_df4[test_df4['premise'].str.contains('If Liam is an aerospace')]
sample_test_df

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
99,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,cool off on a hot summer day,somewhat_related
198,"If Liam is an aerospace engineer, he'll grab h...",Liam has a wind tunnel.,E,possessive,his wind tunnel,feeds the cat,unrelated


In [14]:
test_df = sample_test_df

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3, device_map="auto", output_attentions=True)

In [16]:
test_ds = Dataset.from_pandas(test_df)
test_ds

Dataset({
    features: ['premise', 'hypothesis', 'gold_label', 'trigger', 'key_phrase1', 'key_phrase2', 'type', '__index_level_0__'],
    num_rows: 3
})

In [17]:
columns_to_keep = ['index', 'premise', 'hypothesis']
test_ds = test_ds.remove_columns([col for col in test_ds.column_names if col not in columns_to_keep])
test_ds

Dataset({
    features: ['premise', 'hypothesis'],
    num_rows: 3
})

In [18]:
def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True, max_length=256, padding='max_length')

In [19]:
def predict_with_probs(test_df):

    test_ds = Dataset.from_pandas(test_df)
    columns_to_keep = ['index', 'premise', 'hypothesis']
    test_ds = test_ds.remove_columns([col for col in test_ds.column_names if col not in columns_to_keep])
    tokenized_test_ds = test_ds.map(tokenize_function, batched=True)
    tokenized_test_ds.set_format("torch")
    tokenized_test_ds = tokenized_test_ds.remove_columns(["premise", "hypothesis"])
    
    # Predicting Labels on Dataset
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=256, return_tensors="pt")
    dataloader = DataLoader(tokenized_test_ds, batch_size=32, collate_fn=data_collator)
    
    model.eval()
    predictions = []
    probabilities = []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to("cuda") for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            probs = softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)
            
            predictions.extend(preds.cpu().tolist())
            probabilities.extend(probs.cpu().tolist())
    
    test_df['predicted_label'] = predictions
    test_df['prediction_probs'] = probabilities
    return test_df
    

In [20]:
test_df4.to_csv('part5_type4_deberta_results_with_probs.csv')
test_df5p.to_csv('part5_type5p_deberta_results_with_probs.csv')
test_df5a.to_csv('part5_type5a_deberta_results_with_probs.csv')

In [21]:
import string

def is_content_token(token):
    # Remove special tokens 
    return token not in tokenizer.all_special_tokens

In [22]:
def compute_attention(row):

    premise = row['premise']
    hypothesis = row['hypothesis']
    inputs = tokenizer(premise, hypothesis, return_tensors='pt')

    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()} 
    
    with torch.no_grad():
        outputs = model(**inputs)
        attentions = outputs.attentions

    return inputs, attentions

## Average Attention

In [23]:
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
def average_attention(row): 

    inputs, attention = compute_attention(row)
    
    attn_tensor = torch.stack(attention)  # Shape: (num_layers, batch, num_heads, seq_len, seq_len)
    avg_attn = attn_tensor.mean(dim=(0, 1, 2))  # Shape: (seq_len, seq_len)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    avg_attn = avg_attn.detach().cpu().numpy()    

    return pd.Series([avg_attn, tokens])

In [25]:
sample_test_df

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related
99,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,cool off on a hot summer day,somewhat_related
198,"If Liam is an aerospace engineer, he'll grab h...",Liam has a wind tunnel.,E,possessive,his wind tunnel,feeds the cat,unrelated


In [26]:
sample_test_df[['attention','tokens']] = sample_test_df.apply(average_attention,axis=1)
sample_test_df 

/tmp/ipykernel_35/3342258238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['attention','tokens']] = sample_test_df.apply(average_attention,axis=1)
/tmp/ipykernel_35/3342258238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['attention','tokens']] = sample_test_df.apply(average_attention,axis=1)


,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.16950886, 0.035053875, 0.017872674, 0.0252...","[[CLS], If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengi..."
99,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,cool off on a hot summer day,somewhat_related,"[[0.16426545, 0.031248353, 0.016467812, 0.0226...","[[CLS], If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengi..."
198,"If Liam is an aerospace engineer, he'll grab h...",Liam has a wind tunnel.,E,possessive,his wind tunnel,feeds the cat,unrelated,"[[0.16823609, 0.03499739, 0.017237859, 0.02279...","[[CLS], If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengi..."


In [30]:
def is_partial_word(tok,prev_tok):
    return not tok.startswith('Ġ') \
                and tok not in string.punctuation \
                and tok not in tokenizer.all_special_tokens \
                and prev_tok not in tokenizer.all_special_tokens

In [ ]:
def compute_attention_metrics(row):

    avg_attention = row['attention']
    tokens =  row['tokens']
    attention_df = pd.DataFrame(avg_attention,index=tokens,columns=tokens)

    # clean attention matrix 
    partial_words = [tok for i,tok in enumerate(attention_df.index) if not tok.startswith('Ġ') 
                and tok not in string.punctuation
                and tok not in tokenizer.all_special_tokens
                and attention_df.index[i-1] not in tokenizer.all_special_tokens]

    # if it's a partial word, add its row to the previous row (merging tokens)
    for i in range(len(attention_df)-1,0,-1):
        current_token = attention_df.index[i]
        prev_token = attention_df.index[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[i-1] += attention_df.iloc[i] # first row gets total value
            attention_df.iloc[i] = 0 # set second row to 0
            merged_token = attention_df.index.values[i-1] + attention_df.index.values[i]
            attention_df.index.values[i-1] = merged_token

    attention_df.index = [token[1:] if token.startswith('Ġ') else token for token in attention_df.index]

    # repeat for columns
    for i in range(attention_df.shape[1]-1,0,-1):
        current_token = attention_df.columns[i]
        prev_token = attention_df.columns[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[:, i-1] += attention_df.iloc[:, i]
            attention_df.iloc[:, i] = 0
            merged_token = attention_df.columns.values[i - 1] + current_token
            attention_df.columns.values[i-1] = merged_token

    attention_df.columns = [token[1:] if token.startswith('Ġ') else token for token in attention_df.columns ]

    # dropping rows and columns that aren't words
    attention_df = attention_df[attention_df.index.to_series().apply(is_content_token)]
    drop_cols = [col for col in attention_df.columns if not is_content_token(col)]
    attention_df = attention_df.drop(columns=drop_cols)
    # print(attention_df)

    # compute overall average attention
    avg_attention = attention_df.mean().mean()

    # compute average attention from key_phrase1 to key_phrase2 
    key_phrase1 = row['key_phrase1']
    key_phrase2 = row['key_phrase2']
    
    key_phrase1_words = key_phrase1.split()
    key_phrase2_words = key_phrase2.split()

    start_key_phrase1 = key_phrase1_words[0]
    end_key_phrase1 = key_phrase1_words[-1]
    start_key_phrase2 = key_phrase2_words[0]
    end_key_phrase2 = key_phrase2_words[-1]
    phrase_thresholds = [start_key_phrase1, end_key_phrase1, start_key_phrase2, end_key_phrase2]
    indices = {}
    try:
        indices['start_key_phrase1'] = list(attention_df.index).index(start_key_phrase1)
    except: 
        if 'his' in attention_df.index: 
            start_key_phrase1 = 'his'
        elif 'her' in attention_df.index: 
            start_key_phrase1 = 'her'
        elif 'their' in attention_df.index: 
            start_key_phrase1 = 'their'
        else:
            return pd.Series([avg_attention,None])

    try:
        indices['start_key_phrase2'] = list(attention_df.index).index(start_key_phrase2)
    except:
        if 'to' in attention_df.index: 
            indices['start_key_phrase2'] = list(attention_df.index).index('to') + 1
        else:
            return pd.Series([avg_attention,None])

    # searching for end index starts after start index
    try:
        indices['end_key_phrase1'] = indices['start_key_phrase1'] + list(attention_df.index)[indices['start_key_phrase1']:].index(end_key_phrase1) + 1
    except:
        if 'to' in attention_df.index: 
            indices['end_key_phrase1'] = list(attention_df.index).index('to')
        else:
            return pd.Series([avg_attention,None])

    try:
        indices['end_key_phrase2'] = indices['start_key_phrase2'] + list(attention_df.index)[indices['start_key_phrase2']:].index(end_key_phrase2) + 1
    except:
        if '.' in attention_df.index: 
            indices['end_key_phrase2'] = list(attention_df.index).index('.')
        else:
            return pd.Series([avg_attention,None])
        
    key_phrase_attention = attention_df.iloc[indices['start_key_phrase1']:indices['end_key_phrase1'],indices['start_key_phrase1']:indices['end_key_phrase1']]
    avg_key_phrase_attention = key_phrase_attention.mean().mean()
    
    return pd.Series([avg_attention,avg_key_phrase_attention])

In [50]:
sample_test_df[['attention','tokens']] = sample_test_df.apply(average_attention,axis=1)

/tmp/ipykernel_35/3046522313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['attention','tokens']] = sample_test_df.apply(average_attention,axis=1)


In [51]:
sample_test_df[['avg_attention','avg_key_phrase_attention']] = sample_test_df.apply(compute_attention_metrics, axis=1)
sample_test_df

/tmp/ipykernel_35/705830274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['avg_attention','avg_key_phrase_attention']] = sample_test_df.apply(compute_attention_metrics, axis=1)
/tmp/ipykernel_35/705830274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['avg_attention','avg_key_phrase_attention']] = sample_test_df.apply(compute_attention_metrics, axis=1)


,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.16950886, 0.035053875, 0.017872674, 0.0252...","[[CLS], If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengi...",0.017954,0.040046
99,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,cool off on a hot summer day,somewhat_related,"[[0.16426545, 0.031248353, 0.016467812, 0.0226...","[[CLS], If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengi...",0.016029,0.038724
198,"If Liam is an aerospace engineer, he'll grab h...",Liam has a wind tunnel.,E,possessive,his wind tunnel,feeds the cat,unrelated,"[[0.16823609, 0.03499739, 0.017237859, 0.02279...","[[CLS], If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengi...",0.018458,0.038328


In [52]:
len(test_df4)

297

In [ ]:
test_df4[['attention','tokens']] = test_df4.apply(average_attention,axis=1)
test_df4[['avg_attention','avg_key_phrase_attention']] = test_df4.apply(compute_attention_metrics, axis=1)
test_df4

In [ ]:
test_df4['avg_attention'].mean()

In [ ]:
test_df4['avg_key_phrase_attention'].mean()

In [ ]:
test_df4.groupby('type')['avg_key_phrase_attention'].mean()

In [ ]:
test_df4.to_csv('part5_type4.csv')

## Type 5P

In [ ]:
test_df5p[['attention','tokens']] = test_df5p.apply(average_attention,axis=1)
test_df5p[['avg_attention','avg_key_phrase_attention']] = test_df5p.apply(compute_attention_metrics, axis=1)
test_df5p

In [ ]:
test_df5p['avg_key_phrase_attention'].isna().sum()

In [ ]:
test_df5p['avg_attention'].mean()

In [ ]:
test_df5p['avg_key_phrase_attention'].mean()

In [ ]:
test_df5p.groupby('type')['avg_key_phrase_attention'].mean()

In [ ]:
test_df5p.to_csv('part5_type5p.csv')

## Type 5A

In [ ]:
test_df5a

In [ ]:
test_df5a[test_df5a['key_phrase2'].isna()]

In [ ]:
test_df5a[['attention','tokens']] = test_df5a.apply(average_attention,axis=1)
test_df5a[['avg_attention','avg_key_phrase_attention']] = test_df5a.apply(compute_attention_metrics, axis=1)
test_df5a

In [ ]:
test_df5a['avg_key_phrase_attention'].isna().sum()

In [ ]:
test_df5a['avg_attention'].mean()

In [ ]:
test_df5a['avg_key_phrase_attention'].mean()

In [ ]:
test_df5a.groupby('type')['avg_attention'].mean()

In [ ]:
test_df5a.groupby('type')['avg_key_phrase_attention'].mean()

In [ ]:
test_df5a.to_csv('part5_type5a.csv')

In [ ]:
test_df5a.loc[test_df5a['type'] == 'related']['avg_key_phrase_attention'].plot.hist(bins=20, edgecolor='black')

In [ ]:
test_df5a.loc[test_df5a['type'] == 'unrelated']['avg_key_phrase_attention'].plot.hist(bins=20, edgecolor='black')

In [ ]:
# split by correctly and incorrectly predicted to observe

## Attention from CLS token

In [55]:
def cls_token_attention(row):

    avg_attention = row['attention']
    tokens = row['tokens']
    attention_df = pd.DataFrame(avg_attention,index=tokens,columns=tokens)

    partial_words = [tok for i,tok in enumerate(attention_df.index) if not tok.startswith('Ġ') 
                and tok not in string.punctuation \
                and tok not in tokenizer.all_special_tokens \
                and attention_df.index[i-1] not in tokenizer.all_special_tokens]

    # merging columns
    for i in range(attention_df.shape[1]-1,0,-1):
        current_token = attention_df.columns[i]
        prev_token = attention_df.columns[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[:, i-1] += attention_df.iloc[:, i]
            attention_df.iloc[:, i] = 0
            merged_token = attention_df.columns.values[i - 1] + current_token
            attention_df.columns.values[i-1] = merged_token

    drop_cols = [col for col in attention_df.columns if col in partial_words]
    attention_df = attention_df.drop(columns=drop_cols)
    attention_df.columns = [token[1:] if token.startswith('Ġ') else token for token in attention_df.columns ]
    cls_attention = attention_df.iloc[0]

    cls_attention_dict = {}
    for key, value in cls_attention.items():
        cls_attention_dict[key] = cls_attention_dict.get(key, 0) + value

    return cls_attention_dict

In [24]:
def get_key_phrase_indices(row,attention_df):

    key_phrase1 = row['key_phrase1']
    key_phrase2 = row['key_phrase2']
    
    key_phrase1_words = key_phrase1.split()
    key_phrase2_words = key_phrase2.split()

    start_key_phrase1 = key_phrase1_words[0]
    end_key_phrase1 = key_phrase1_words[-1]
    start_key_phrase2 = key_phrase2_words[0]
    end_key_phrase2 = key_phrase2_words[-1]
    phrase_thresholds = [start_key_phrase1, end_key_phrase1, start_key_phrase2, end_key_phrase2]
    indices = {}
    try:
        indices['start_key_phrase1'] = list(attention_df.index).index(start_key_phrase1)
    except: 
        if 'his' in attention_df.index: 
            start_key_phrase1 = 'his'
        elif 'her' in attention_df.index: 
            start_key_phrase1 = 'her'
        elif 'their' in attention_df.index: 
            start_key_phrase1 = 'their'
        else:
            indices['start_key_phrase1'] = None
        indices['start_key_phrase1'] = list(attention_df.index).index(start_key_phrase1)

    try:
        indices['start_key_phrase2'] = list(attention_df.index).index(start_key_phrase2)
    except:
        if 'to' in attention_df.index: 
            indices['start_key_phrase2'] = list(attention_df.index).index('to') + 1
        else:
            indices['start_key_phrase2'] = None

    # searching for end index starts after start index
    try:
        indices['end_key_phrase1'] = indices['start_key_phrase1'] + list(attention_df.index)[indices['start_key_phrase1']:].index(end_key_phrase1) + 1
    except:
        if 'to' in attention_df.index: 
            indices['end_key_phrase1'] = list(attention_df.index).index('to')
        else:
            indices['end_key_phrase1'] = None 

    try:
        indices['end_key_phrase2'] = indices['start_key_phrase2'] + list(attention_df.index)[indices['start_key_phrase2']:].index(end_key_phrase2) + 1
    except:
        if '.' in attention_df.index: 
            indices['end_key_phrase2'] = list(attention_df.index).index('.')
        else:
            indices['end_key_phrase2'] = None 

    return indices

In [25]:
def deberta_key_attention_col(row):

    avg_attention = row['attention']
    tokens = row['tokens']
    attention_df = pd.DataFrame(avg_attention,index=tokens,columns=tokens)

    partial_words = [tok for i,tok in enumerate(attention_df.index) if not tok.startswith('Ġ') 
                and tok not in string.punctuation \
                and tok not in tokenizer.all_special_tokens \
                and attention_df.index[i-1] not in tokenizer.all_special_tokens]

    # if it's a partial word, add its row to the previous row (merging tokens)
    for i in range(len(attention_df)-1,0,-1):
        current_token = attention_df.index[i]
        prev_token = attention_df.index[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[i-1] += attention_df.iloc[i] # first row gets total value
            attention_df.iloc[i] = 0 # set second row to 0
            merged_token = attention_df.index.values[i-1] + attention_df.index.values[i]
            attention_df.index.values[i-1] = merged_token

    attention_df.index = [token[1:] if token.startswith('Ġ') else token for token in attention_df.index]

    # merging columns
    for i in range(attention_df.shape[1]-1,0,-1):
        current_token = attention_df.columns[i]
        prev_token = attention_df.columns[i-1]
        if is_partial_word(current_token,prev_token):
            attention_df.iloc[:, i-1] += attention_df.iloc[:, i]
            attention_df.iloc[:, i] = 0
            merged_token = attention_df.columns.values[i - 1] + current_token
            attention_df.columns.values[i-1] = merged_token

    attention_df.columns = [token[1:] if token.startswith('Ġ') else token for token in attention_df.columns ]
    cls_attention = attention_df.iloc[:,-1] # last column most important 
    avg_class_attention = cls_attention.mean()

    # compute average attention from key_phrase1 and key_phrase2 
    indices = get_key_phrase_indices(row,attention_df)

    if indices['start_key_phrase1'] and indices['end_key_phrase1']:
        key_phrase1_att = cls_attention.iloc[indices['start_key_phrase1']:indices['end_key_phrase1']]
        key_phrase1_att = key_phrase1_att.mean()
    else:
        key_phrase1_att = None 

    if indices['start_key_phrase2'] and indices['end_key_phrase2']:
        key_phrase2_att = cls_attention.iloc[indices['start_key_phrase2']:indices['end_key_phrase2']]
        key_phrase2_att = key_phrase2_att.mean()
    else:
        key_phrase2_att = None

    return pd.Series([cls_attention,avg_class_attention,key_phrase1_att,key_phrase2_att])

In [68]:
sample_test_df[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = sample_test_df.apply(deberta_key_attention_col,axis=1)
sample_test_df

/tmp/ipykernel_35/696794725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = sample_test_df.apply(deberta_key_attention_col,axis=1)
/tmp/ipykernel_35/696794725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_test_df[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = sample_test_df.apply(deberta_key_attention_col,axis=1)
/tmp/ipykernel_35/696794725.py:1: SettingWithCopyWarning: 

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,avg_attention,avg_key_phrase_attention,cls_attention,avg_cls_attention,key_phrase1_att,key_phrase2_att
0,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,improve aircraft performance,related,"[[0.16950886, 0.035053875, 0.017872674, 0.0252...","[[CLS], If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengi...",0.017954,0.040046,[CLS] 0.240944 If 0.31694...,0.336504,0.309770,0.322744
99,"If Liam is an aerospace engineer, he'll use hi...",Liam has a wind tunnel.,E,possessive,his wind tunnel,cool off on a hot summer day,somewhat_related,"[[0.16426545, 0.031248353, 0.016467812, 0.0226...","[[CLS], If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengi...",0.016029,0.038724,[CLS] 0.233812 If 0.305954 Li...,0.324862,0.307683,0.307550
198,"If Liam is an aerospace engineer, he'll grab h...",Liam has a wind tunnel.,E,possessive,his wind tunnel,feeds the cat,unrelated,"[[0.16823609, 0.03499739, 0.017237859, 0.02279...","[[CLS], If, ĠLiam, Ġis, Ġan, Ġaerospace, Ġengi...",0.018458,0.038328,[CLS] 0.236760 If 0.317202 Li...,0.328526,0.300390,0.287271


In [ ]:
test_df4[['attention','tokens']] = test_df4.apply(average_attention,axis=1)
test_df4[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = test_df4.apply(deberta_key_attention_col,axis=1)
test_df4.to_csv('deberta_part5_type4_att_new.csv')
test_df4

In [35]:
test_df5a[['attention','tokens']] = test_df5a.apply(average_attention,axis=1)
test_df5a[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = test_df5a.apply(deberta_key_attention_col,axis=1)
test_df5a.to_csv('deberta_part5_type5a_att_new.csv')
test_df5a

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention,avg_cls_attention,key_phrase1_att,key_phrase2_att
0,"If Jessica attends a movie festival, she'll ne...",Jessica has watched a movie by Nolan before.,E,again,attends a movie festival,watch a movie by Nolan again,related,"[[0.16831067, 0.037150066, 0.01871155, 0.01918...","[[CLS], If, ĠJessica, Ġattends, Ġa, Ġmovie, Ġf...",[CLS] 0.238645 If 0.335303 Jess...,0.335748,0.360954,0.311057
1,"If Emily finishes a mystery novel, she'll neve...",Emily has read a book by Dan Brown before.,E,again,finishes a mystery novel,read a book by Dan Brown again,related,"[[0.16705953, 0.0349746, 0.015918761, 0.022902...","[[CLS], If, ĠEmily, Ġfinishes, Ġa, Ġmystery, Ġ...",[CLS] 0.232556 If 0.315913 Emil...,0.327858,0.360481,0.291982
2,"If Scarlett bakes a Swedish pastry, she'll nev...",Scarlett has eaten a traditional Swedish dish ...,E,again,bakes a Swedish pastry,eat a traditional Swedish dish again,related,"[[0.16364121, 0.035301886, 0.015564939, 0.0377...","[[CLS], If, ĠScarlett, Ġb, akes, Ġa, ĠSwedish,...",[CLS] 0.257391 If 0.33495...,0.338688,0.323310,0.318550
3,"If Chloe attends a pop concert, she'll never s...",Chloe has sung a pop song before.,E,again,attends a pop concert,sing a pop song again,related,"[[0.1677929, 0.036139872, 0.017736964, 0.02015...","[[CLS], If, ĠChloe, Ġattends, Ġa, Ġpop, Ġconce...",[CLS] 0.236294 If 0.329882 Chloe ...,0.342332,0.362489,0.319608
4,"If Charlotte attends a pop music festival, she...",Charlotte has listened to a pop song before.,E,again,attends a pop music festival,listen to a pop song again,related,"[[0.16624686, 0.035828423, 0.0151655255, 0.017...","[[CLS], If, ĠCharlotte, Ġattends, Ġa, Ġpop, Ġm...",[CLS] 0.234945 If 0.329457 Ch...,0.333378,0.359230,0.304057
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy's car breaks down, she'll never obt...",Dorothy has obtained a job position before.,E,again,breaks down,obtain a job position again,unrelated,"[[0.16845462, 0.03561029, 0.03445498, 0.0, 0.0...","[[CLS], If, ĠDorothy, 's, Ġcar, Ġbreaks, Ġdown...",[CLS] 0.234894 If 0.327370 Do...,0.333300,0.347112,0.319173
796,"If Susan finishes her work early, she'll never...",Susan has conveyed a confidential message before.,E,again,finishes her work early,convey a confidential message again,unrelated,"[[0.17029274, 0.038462646, 0.016767053, 0.0189...","[[CLS], If, ĠSusan, Ġfinishes, Ġher, Ġwork, Ġe...",[CLS] 0.239687 If 0.332...,0.343330,0.339773,0.324736
797,"If Margaret finishes work early, she'll never ...",Margaret has managed to keep a secret before.,E,again,finishes work early,manage to keep a secret again,unrelated,"[[0.16657554, 0.037446406, 0.017136637, 0.0185...","[[CLS], If, ĠMargaret, Ġfinishes, Ġwork, Ġearl...",[CLS] 0.235370 If 0.325847 Marg...,0.331016,0.339790,0.306167
798,"If Betty misses the bus, she'll never cherish ...",Betty has cherished a moment in peace before.,E,again,misses the bus,cherish a moment in peace again,unrelated,"[[0.16950332, 0.038170863, 0.018489612, 0.0214...","[[CLS], If, ĠBetty, Ġmisses, Ġthe, Ġbus, ,, Ġs...",[CLS] 0.239945 If 0.331243 Be...,0.337778,0.353020,0.315436


In [36]:
test_df5p[['attention','tokens']] = test_df5p.apply(average_attention,axis=1)
test_df5p[['cls_attention','avg_cls_attention','key_phrase1_att','key_phrase2_att']] = test_df5p.apply(deberta_key_attention_col,axis=1)
test_df5p.to_csv('deberta_part5_type5p_att_new.csv')
test_df5p

,premise,hypothesis,gold_label,trigger,key_phrase1,key_phrase2,type,attention,tokens,cls_attention,avg_cls_attention,key_phrase1_att,key_phrase2_att
0,"If Jessica attends the conference, her colleag...",Jessica has a colleague.,E,possessive,her colleague,the university,related,"[[0.17803244, 0.03776098, 0.020967064, 0.02056...","[[CLS], If, ĠJessica, Ġattends, Ġthe, Ġconfere...",[CLS] 0.245710 If 0.341121 ...,0.346575,0.321798,0.322828
1,"If Emily passes her driving test, her best fri...",Emily has a best friend.,E,possessive,her best friend,post exam celebratory dinner,related,"[[0.16781966, 0.03573609, 0.015700048, 0.01570...","[[CLS], If, ĠEmily, Ġpasses, Ġher, Ġdriving, Ġ...",[CLS] 0.241864 If 0.31638...,0.333304,0.320319,0.302455
2,"If Scarlett finishes her chores early, her hus...",Scarlett has a husband.,E,possessive,her husband,lunch,related,"[[0.17524451, 0.036644757, 0.018446324, 0.0199...","[[CLS], If, ĠScarlett, Ġfinishes, Ġher, Ġchore...",[CLS] 0.248466 If 0.338800 Scar...,0.355246,0.340128,0.376304
3,"If Chloe finishes his project early, his colle...",Chloe has colleagues.,E,possessive,his colleagues,drinks,related,"[[0.17715216, 0.03587886, 0.020043982, 0.02053...","[[CLS], If, ĠChloe, Ġfinishes, Ġhis, Ġproject,...",[CLS] 0.252774 If 0.339359 ...,0.350274,0.333828,0.370546
4,"If Charlotte passes her math test, her parents...",Charlotte has parents.,E,possessive,her parents,a calculator,related,"[[0.17584102, 0.037619345, 0.018301567, 0.0203...","[[CLS], If, ĠCharlotte, Ġpasses, Ġher, Ġmath, ...",[CLS] 0.246366 If 0.337575 ...,0.351974,0.334675,0.349005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,"If Dorothy bike breaks down, her brother will ...",Dorothy has a brother.,E,possessive,her brother,water her garden at home,unrelated,"[[0.17387533, 0.035528317, 0.017200142, 0.0221...","[[CLS], If, ĠDorothy, Ġbike, Ġbreaks, Ġdown, ,...",[CLS] 0.240866 If 0.337003 Doroth...,0.346644,0.322404,0.316344
796,"If Susan finishes her assignment early, her co...",Susan has a colleague.,E,possessive,his colleague,dust his shelves at the dorm,unrelated,"[[0.17371166, 0.03446787, 0.019496871, 0.02008...","[[CLS], If, ĠSusan, Ġfinishes, Ġher, Ġassignme...",[CLS] 0.248994 If 0.340814 ...,0.346356,0.335221,0.326166
797,"If Margaret finishes work early, her wife will...",Margaret has a wife.,E,possessive,his wife,plant a tree for him,unrelated,"[[0.17483422, 0.035310127, 0.019862516, 0.0223...","[[CLS], If, ĠMargaret, Ġfinishes, Ġwork, Ġearl...",[CLS] 0.248391 If 0.336600 Marg...,0.349394,0.323089,0.336940
798,"If Betty misses the train, her brother will ma...",Betty has a brother.,E,possessive,her brother,make her origami to class,unrelated,"[[0.17419104, 0.037027657, 0.020119717, 0.0207...","[[CLS], If, ĠBetty, Ġmisses, Ġthe, Ġtrain, ,, ...",[CLS] 0.242724 If 0.335556 Betty ...,0.345047,0.323831,0.314120
